# Analiza tekem ATP World Tour

V tem zvezku bomo analizirali rezultate vseh profesionalnih teniških tekem za posameznega moškega tekmovalca v rangu od 1. do 300., za leto 2018. V osnovi nas zanima, če je možno, glede na pridobljene podatke o lastnostnih posameznih tekmovalcev v eni tekmi, realno napovedati rezultat tekme. Lastnosti, katere imamo namen pridobiti so: 
- rang igralca
- telesna višina, 
- telesna teža, 
- starost, 
- primarna  igrana roka, 
- tip igrišča

Nad temi podatki bomo prožili klasifikacijo in rezultate primerjali z dejanskimi rezultati odigranih tekem.

## Pridobivanje podatkov

Pred samo analizo vsebine moramo te podatke najprej pridobiti iz spletne strani [ATP World Tour](http://www.atpworldtour.com/). To smo storili s pomočjo Python knjižnice [Scrapy](https://scrapy.org/).

Podatke bomo brali iz strani kot je ta: [https://www.atpworldtour.com/en/players/roger-federer/f324/player-activity](https://www.atpworldtour.com/en/players/roger-federer/f324/player-activity)

<img src="atp_activity.png">

Iz strani bomo najprej prebrali vse relevantne podatke za primarnega igralca, nato pa še podatke o vsakem posameznem nasprotniku iz nasprotnikove profile strani, saj vsi za nas relevantni podatki niso na voljo na prvotni strani.

Podatki, ki jih po koncu branje pridobimo so:
- **court_type**: Tip igrišča (zunanje igrišče ali dvorana)
- **court_base**: Podlaga (pesek, trda podlaga ali tepih)
- **ranking_difference**: Razlika na lestvici med igralcema
- **age_difference**: Razlika v letih med igralcema
- **height_difference**: Razlika v višini med igralcema
- **weight_difference**: Razlika v teži med igralcema
- **player1_ranking**: Razvrstiev primarnega igralca
- **player2_ranking**: Razvrstitev nasprotnika
- **player1_age**: Starost primarnega igralca
- **player2_age**: Starost nasprotnika
- **player1_height**: Višina primarnega igralca
- **player2_height**: Višina nasprotnika
- **player1_weight**: Teža primarnega igralca
- **player2_weight**: Teža nasprotnika
- **player1_birthplace**: Rojstna država primarnega igralca
- **player2_birthplace**: Rojstna država nasprotnika
- **player1_main_hand**: Glavna roka primarnega igralca
- **player2_main_hand**: Glavna roka nasprotnika
- **player1_backhand**: Backhand udarec primarnega igralca
- **player2_backhand**: Backhand nasprotnika
- **result**: Rezultat dvoboja

Po samem branju ustvarimo datoteko in vanjo shranimo prebrane podatke. Tako dobimo datoteko [atp.csv](atp3.csv)

## Branje podatkov

Za branje in pripravo za nadaljnjo uporabo bomo uporabili knjižnico [Pandas](https://pandas.pydata.org/). Za indeksiran stolpec smo že vnaprej, med "strganjem", pripravili poseben stolpec, kateri je po vrsti štel vnešene podatke.

Po branju izbrišemo vrstice, katere vsebujejo ničelne podatke.

In [35]:
import pandas as pd

df = pd.read_csv("atp3.csv", index_col=6, sep=',', decimal=".")
df = df.dropna(axis=0)

df.head()

,p1_age,p1_birthplace,p1_main_hand,p1_backhand,court_type,court_base,p1_ranking,p2_ranking,ranking_diff,result,p2_age,p2_birthplace,p2_main_hand,p2_backhand,age_diff,weight_diff,height_diff
activity_id,,,,,,,,,,,,,,,,,
0,32.0,Spain,Left-Handed,Two-Handed Backhand,O,Clay,1,8,7,W,37,DominicanRepublic,Right-Handed,One-Handed Backhand,8.0,3.0,26.0
1,32.0,Spain,Left-Handed,Two-Handed Backhand,O,Clay,1,6,5,W,26,Germany,Right-Handed,Two-Handed Backhand,3.0,6.0,3.0
2,32.0,Spain,Left-Handed,Two-Handed Backhand,O,Clay,1,12,11,W,18,Hungary,Right-Handed,Two-Handed Backhand,16.0,12.0,0.0
3,32.0,Spain,Left-Handed,Two-Handed Backhand,O,Clay,1,70,69,W,25,USA,Right-Handed,Two-Handed Backhand,9.0,1.0,13.0
4,32.0,Spain,Left-Handed,Two-Handed Backhand,O,Clay,1,32,31,W,33,France,Right-Handed,Two-Handed Backhand,1.0,7.0,5.0


## Obdelava podatkov

Ker knjižnica [scikit-learn](http://scikit-learn.org/stable/index.html) (sklearn) ne zna obdelati nominalnih podatkov, bomo vse neštevilske podatke prevtorili v številske. Za to bomo uporabili **indikacijske parametre**.

In [36]:
nominal_data = ["p1_main_hand", "p2_main_hand", "court_type", "court_base" ]
df = pd.get_dummies(df, columns=nominal_data)

df.head()

,p1_age,p1_birthplace,p1_backhand,p1_ranking,p2_ranking,ranking_diff,result,p2_age,p2_birthplace,p2_backhand,...,p1_main_hand_Ambidextrous,p1_main_hand_Left-Handed,p1_main_hand_Right-Handed,p2_main_hand_Left-Handed,p2_main_hand_Right-Handed,court_type_I,court_type_O,court_base_Carpet,court_base_Clay,court_base_Hard
activity_id,,,,,,,,,,,,,,,,,,,,,
0,32.0,Spain,Two-Handed Backhand,1,8,7,W,37,DominicanRepublic,One-Handed Backhand,...,0,1,0,0,1,0,1,0,1,0
1,32.0,Spain,Two-Handed Backhand,1,6,5,W,26,Germany,Two-Handed Backhand,...,0,1,0,0,1,0,1,0,1,0
2,32.0,Spain,Two-Handed Backhand,1,12,11,W,18,Hungary,Two-Handed Backhand,...,0,1,0,0,1,0,1,0,1,0
3,32.0,Spain,Two-Handed Backhand,1,70,69,W,25,USA,Two-Handed Backhand,...,0,1,0,0,1,0,1,0,1,0
4,32.0,Spain,Two-Handed Backhand,1,32,31,W,33,France,Two-Handed Backhand,...,0,1,0,0,1,0,1,0,1,0


## Klasifikacija

Podatke smo pretvorili v številske in jih s tem pripravili na učenje. 

Na tem mestu moramo ločiti atribute nad katerimi se bomo učili. Uporabili bomo po večini vse, razen dejanskih let, višine teže in države rojstva posameznega igralca. Za naš model bodo bolj pomembne podatki predstavljeni v razliki zgoraj omenjenih atributov.

Za drugi del določimo izhod, ki nas zanima. V našem primeru je to rezultat dvoboja.

S tem smo podatke pripravili za učenje z učnim algoritmom knjižnice **sklearn**. Za ućenje smo uporabili učenje **odločitvenega klasifikacijskega drevesa (angl. [Descision Tree Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html))**. Naša testna množica bo velika 25% vseh podatkov, učna pa 75%. 

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

input = [
    "ranking_diff", 
    "age_diff", 
    "weight_diff", 
    "height_diff", 
    "p1_main_hand_Ambidextrous",
    "p1_main_hand_Left-Handed", 
    "p1_main_hand_Right-Handed", 
    "p2_main_hand_Left-Handed", 
    "p2_main_hand_Right-Handed",
    "court_type_I", 
    "court_type_O", 
    "court_base_Carpet", 
    "court_base_Clay", 
    "court_base_Hard"
]
output = "result"

x_train, x_test, y_train, y_test = train_test_split(df[input], df[output], test_size=0.25)

dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Testiranje

Z naučenim modelom lahko sedaj poskusimo pravilno napovedati rezultat dvoboja med dvema igralcema. 

In [42]:
import numpy as np

player1_rank = 8
player1_age = 30
player1_weight = 90
player1_height = 189
player1_main_hand_ambidextrous = 1
player1_main_hand_right_handed = 0
player1_main_hand_left_handed = 0

player2_rank = 2
player2_age = 22
player2_weight = 87
player2_height = 171
player2_main_hand_right_handed = 0
player2_main_hand_left_handed = 0

court_type_inside = 0
court_type_outside = 1

court_base_carpet = 0
court_base_clay = 0
court_base_hard = 1

activity = np.array([
    (player1_rank - player2_rank), 
    (player1_age - player2_age), 
    (player1_weight - player2_weight), 
    (player1_height - player2_height), 
    player1_main_hand_ambidextrous,
    player1_main_hand_right_handed,
    player1_main_hand_left_handed,
    player2_main_hand_right_handed,
    player2_main_hand_left_handed,
    court_type_inside,
    court_type_outside,
    court_base_carpet,
    court_base_clay,
    court_base_hard
]).reshape(1, -1)

dt.predict(activity)

array(['W'], dtype=object)

## Vizualizacija

Za vizualizacijo zbranih podatkov bomo uporabili knjižnico [seaborn](https://seaborn.pydata.org/).

In [43]:
import seaborn as sns
sns.set()

## Zaključek